In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import json
import pandas as pd
from datetime import datetime

In [ ]:
# Extract data from Alpha Vantage API
#
#url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=GR67BV7YDLZ4P4B1'
#r = requests.get(url)
#data = r.json()

#print(data)


In [4]:
url = "https://alpha-vantage.p.rapidapi.com/query"

# Set up the query parameters
querystring = {
    "datatype": "json",
    "output_size": "compact",
    "interval": "5min",
    "function": "TIME_SERIES_DAILY",
    "symbol": "MSFT"
}

# headers using environment variables
headers = {
    "x-rapidapi-key": os.getenv("RAPIDAPI_KEY"),
    "x-rapidapi-host": "alpha-vantage.p.rapidapi.com"
}

# Fetch data
response = requests.get(url, headers=headers, params=querystring)

# Check for a successful response
if response.status_code != 200:
    raise Exception(f"API request failed: {response.status_code}")
data = response.json() # Parse JSON
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'MSFT',
  '3. Last Refreshed': '2025-10-03',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2025-10-03': {'1. open': '517.1000',
   '2. high': '520.4900',
   '3. low': '515.0000',
   '4. close': '517.3500',
   '5. volume': '15112321'},
  '2025-10-02': {'1. open': '517.6400',
   '2. high': '521.6000',
   '3. low': '510.6791',
   '4. close': '515.7400',
   '5. volume': '21222886'},
  '2025-10-01': {'1. open': '514.8000',
   '2. high': '520.5050',
   '3. low': '511.6900',
   '4. close': '519.7100',
   '5. volume': '22632336'},
  '2025-09-30': {'1. open': '513.2400',
   '2. high': '518.1600',
   '3. low': '509.6600',
   '4. close': '517.9500',
   '5. volume': '19728229'},
  '2025-09-29': {'1. open': '511.5000',
   '2. high': '516.8450',
   '3. low': '508.8800',
   '4. close': '514.6000',
   '5. volume': '17617775'},
  '2025-09-26': {'1. open': '510

In [9]:
# Extract time series data
symbol = querystring["symbol"]
timeseries_key = next((key for key in data if "Time Series" in key), None)
timeseries = data.get(timeseries_key, {})


stock_data_df = pd.DataFrame.from_dict(timeseries, orient="index")
stock_data_df.columns = ["open", "high", "low", "close", "volume"]
stock_data_df.index = pd.to_datetime(stock_data_df.index)
stock_data_df = stock_data_df.sort_index()

# Add metadata
stock_data_df["symbol"] = symbol
stock_data_df["ingested_at"] = datetime.utcnow()

print(stock_data_df.columns)
print(stock_data_df.head())

Index(['open', 'high', 'low', 'close', 'volume', 'symbol', 'ingested_at'], dtype='object')
                         open      high       low     close  volume symbol  \
2025-10-03 11:40:00  519.5650  519.7000  519.2750  519.6250   94840   MSFT   
2025-10-03 11:45:00  519.5900  519.8100  519.4150  519.8100   69124   MSFT   
2025-10-03 11:50:00  519.8100  520.0300  519.6800  519.7835   99089   MSFT   
2025-10-03 11:55:00  519.7600  519.8800  519.5500  519.6450   73446   MSFT   
2025-10-03 12:00:00  519.6921  520.1300  519.6900  520.0300  113735   MSFT   

                                   ingested_at  
2025-10-03 11:40:00 2025-10-04 17:01:10.380963  
2025-10-03 11:45:00 2025-10-04 17:01:10.380963  
2025-10-03 11:50:00 2025-10-04 17:01:10.380963  
2025-10-03 11:55:00 2025-10-04 17:01:10.380963  
2025-10-03 12:00:00 2025-10-04 17:01:10.380963  


In [10]:
stock_data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2025-10-03 11:40:00 to 2025-10-03 19:55:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   open         100 non-null    object        
 1   high         100 non-null    object        
 2   low          100 non-null    object        
 3   close        100 non-null    object        
 4   volume       100 non-null    object        
 5   symbol       100 non-null    object        
 6   ingested_at  100 non-null    datetime64[us]
dtypes: datetime64[us](1), object(6)
memory usage: 6.2+ KB
